# Reconstructions.ipynb

### Given Bilby parameter estimation results, plot reconstructions using the surrogate model

Maria Okounkova (mokounkova@flatironinstitute.org)

In [1]:
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import corner
from astropy import constants as const
from gwpy.timeseries import TimeSeries
from pycbc.detector import Detector
import bilby
from pesummary.io import read
from math import pi
import h5py
from scipy.interpolate import InterpolatedUnivariateSpline
import lal
import lalsimulation as lalsim
import pycbc
from pycbc.filter.matchedfilter import overlap
from scipy.optimize import minimize_scalar
from ipynb.fs.full.Auxiliary import ReadExtrapolatedModes, EvaluateSurrogate, SubtractPeakTime, dt_eval, df_eval, \
                                    PadAndProject, ProjectGivenGeocenterTime, EllinKm, ComputeSNR, ComputeMultiDetectorSNR, \
                                    ComputeMultiDetectorOverlap, ComputeOverlap, TargetSNR
import gwsurrogate
sur = gwsurrogate.LoadSurrogate('NRSur7dq4')

# Plot parameters
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['axes.labelsize'] = 24
matplotlib.rcParams['xtick.labelsize'] = 30
matplotlib.rcParams['ytick.labelsize'] = 30
matplotlib.rcParams['xtick.major.size'] = 20
matplotlib.rcParams['ytick.major.size'] = 20
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['xtick.minor.visible'] = True
matplotlib.rcParams['xtick.minor.size'] = 10
matplotlib.rcParams['ytick.minor.size'] = 10
matplotlib.rcParams['legend.fontsize'] = 18
matplotlib.rcParams['legend.frameon'] = True
matplotlib.rcParams['lines.linewidth'] = 3

/home/maria.okounkova/.local/lib/python3.6/site-packages/gwtools/rotations.py:63: UserWarning: Could not import GWFrames, needed for rotations module
  _warnings.warn("Could not import GWFrames, needed for rotations module")
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwtools/__init__.py:11: UserWarning: Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.
  _warnings.warn("Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.")


setting __package__ to gwsurrogate.new so relative imports work
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new
setting __package__ to gwsurrogate.new so relative imports work
setting __package__ to gwsurrogate.new so relative imports work


/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_surrogate.py:245: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.t = h5file['t_ds'].value
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_surrogate.py:272: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  'coefs': group['%s_coefs'%(key)].value,
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_surrogate.py:273: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  'bfOrders': group['%s_bfOrders'%(key)].value
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_surrogate.py:282: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  'coefs': group['%s_%d_coefs'%(key, i)].value,
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_sur

Loaded NRSur7dq4 model


### Methods for parameter conversions -- masses, spins, inclination

In [2]:
def EffectiveSpin(params_dict):
    """ Compute effective spin given a parameter dictionary """
    m_1 = params_dict['m_1']
    m_2 = params_dict['m_2']
    a_1 = params_dict['a_1'][2] ## grab z component of spin
    a_2 = params_dict['a_2'][2]
    return (a_1/m_1 + a_2/m_2)/(m_1 + m_2)

def MassParameters(BilbyRes):
    """ Compute individual masses and total mass given bilby result """
    q = 1.0/BilbyRes['mass_ratio']
    chirp = BilbyRes['chirp_mass']
    mass = chirp * (q/(1+q)**2)**(-3/5)
    m_2 = mass / (1 + q)
    m_1 = q*m_2
    return m_1, m_2, mass

def change_spin_convention(theta_jn, phi_jl, tilt1, tilt2, phi12, a1, a2, m1,
                           m2, f_ref, phi_orb=0.):
    """ Change spin convention for Bilby / lal quantities to surrogate model quantities """
    iota, S1x, S1y, S1z, S2x, S2y, S2z = lalsim.SimInspiralTransformPrecessingNewInitialConditions(
                                             theta_jn, phi_jl, tilt1, tilt2, phi12, a1, a2,
                                             m1*lal.MSUN_SI, m2*lal.MSUN_SI, f_ref, phi_orb)
    spin1 = [S1x, S1y, S1z]
    spin2 = [S2x, S2y, S2z]
    return spin1, spin2, iota

def Compute_Spins(bilby_result):
    """ Given a bilby result, compute chi_1 and chi_2, as well as the inclination angle in the form 
        to be passed to a surrogate model """
    
    theta_jn = bilby_result['theta_jn']
    phi_jl = bilby_result['phi_jl']
    tilt1 = bilby_result['tilt_1']
    tilt2 = bilby_result['tilt_2']
    phi12 = bilby_result['phi_12']
    a1 = bilby_result['a_1']
    a2 = bilby_result['a_2']
    m1, m2, mass = MassParameters(bilby_result)
    phi_orb = bilby_result['phase'] ## phase of coalescence at some fiducial time 
    f_ref = 25 ## for now
    spin1, spin2, iota = change_spin_convention(theta_jn, phi_jl, tilt1, tilt2, phi12, a1, a2, m1,
                           m2, f_ref, phi_orb)
    return spin1, spin2, iota

### Methods to read waveform from frames files and read injected parameters

In [3]:
def GetFramesData(p):
    
    det = 'H1'
    sH = TimeSeries.read(source = p + '/Frames/' + det + '.gwf', channel=det+":LDAS_STRAIN")
    tH = np.array(sH.times)
    
    det = 'L1'
    sL = TimeSeries.read(source = p + '/Frames/' + det + '.gwf', channel=det+":LDAS_STRAIN")
    tL = np.array(sH.times)
    
    return tH, sH, tL, sL

def GetInjectedParameters(p):
    """ Return a dictionary of values from the parameters.json file 
        corresponding to the frames, where p is the path to the file """
    
    f = open(p + '/parameters.json')
    params = json.load(f)
    f.close()
    return params

### Methods for reading Bilby parameter estimation results

In [4]:
def GetBilbyParameters(p):
    """ Obtain max likelihood Bilby parameters given what the Bilby Slack said """

    result = bilby.core.result.read_in_result(p + "/outdir/fast_tutorial_result.json")
    maxL_ind = np.argmax(np.array(result.posterior['log_likelihood']))
    maxL_point = result.posterior.iloc[maxL_ind]
    maxP_ind = np.argmax(np.array(result.posterior['log_likelihood'] + result.posterior['log_prior']))
    maxP_point = result.posterior.iloc[maxP_ind]

    return maxP_point

def GetMultipleBilbyParameters(p, n_points):
    """ Obtain n_points max likelihood Bilby parameters """

    result = bilby.core.result.read_in_result(p + "/outdir/fast_tutorial_result.json")
    values = np.array(result.posterior['log_likelihood'] + result.posterior['log_prior'])
    
    sorted_indices = (values.argsort()[::-1])[0:n_points]
    result_points = [result.posterior.iloc[index] for index in sorted_indices]

    return result_points

def GetBilbyDict(BilbyRes):
    """ Convert a Bilby max likelihood result into a dictionary of parameters 
        that we can pass into the surrogate model """
    
    BilbyDict = {}
    BilbyDict['dt'] = 1.0/2048.
    
    ## Reciprocal of the mass ratio 
    q = 1.0/BilbyRes['mass_ratio']
    BilbyDict['q'] = q
    
    m_1, m_2, mass = MassParameters(BilbyRes)
    BilbyDict['mass'] = mass
    BilbyDict['m_1'] = m_1
    BilbyDict['m_2'] = m_2
    
    BilbyDict['dist_mpc'] = BilbyRes['luminosity_distance'] 
    BilbyDict['f_low'] = 0.0
    BilbyDict['phi'] = BilbyRes['phase'] ## phase of coalescence (at some fiducial time)
    
    ## Spin computations
    chi_1, chi_2, theta = Compute_Spins(BilbyRes)

    BilbyDict['a_1'] = chi_1
    BilbyDict['a_2'] = chi_2
    chi_eff = EffectiveSpin(BilbyDict)
    BilbyDict['chi_eff'] = chi_eff
    
    BilbyDict['theta'] = theta ## Theta from the spin computation conventions
    
    BilbyDict['ra'] = BilbyRes['ra']
    BilbyDict['dec'] = BilbyRes['dec']
    BilbyDict['geocent_time'] = BilbyRes['geocent_time']
    BilbyDict['pol'] = BilbyRes['psi'] ## polarization (angle on the sky of the pericenter wrt to interferometer -- degenerate for circular orbits
    
    return BilbyDict


### Method for optimizing the phase between the injected waveform and the reconstructed surrogate model, or finding the optimal f_ref for the surrogate, which we are allowed to freely specify

In [59]:
def OptimizePhase(p, param_dict):
    """ Given a time and a strain array in each detector that we want to compare against, and a dictionary of parameters, 
        iterate over possible phases for the surrogate in order to produce the phase with the largest
        overlap """
    
    timeH, strainH, timeL, strainL = GetFramesData(p)
    
    def f(x):
        surrogate_dict = param_dict
        surrogate_dict['phi'] = x
        time_sur, h_plus_sur, h_cross_sur, amp_sur = EvaluateSurrogate(sur, surrogate_dict)
        timeH_sur, strainH_sur, timeL_sur, strainL_sur = ProjectGivenGeocenterTime(time_sur, h_plus_sur, h_cross_sur, surrogate_dict)
        #ov = ComputeMultiDetectorOverlap(timeH, strainH, timeL, strainL, \
        #                        timeH_sur, strainH_sur, timeL_sur, strainL_sur)
        ov = ComputeOverlap(timeH, strainH, timeH_sur, strainH_sur)
    
        ## 1 minus the overlap since we're doing minimization
        print(1.0 - ov),
        return 1.0 - ov
    
    ## iterate from zero to 2 * pi
    res = minimize_scalar(f, bounds=(-pi/2, 2*pi + pi/2), method = 'bounded')
    return(res.x)

def OptimizeFref(p, param_dict):
    """ Given a time and a strain array in each detector that we want to compare against, and a dictionary of parameters, 
        iterate over possible f_reffor the surrogate in order to produce the phase with the largest
        overlap, since we are allowed to specify f_ref per the surrogate documentation """
    
    timeH, strainH, timeL, strainL = GetFramesData(p)
    
    def f(x):
        surrogate_dict = param_dict
        surrogate_dict['f_ref'] = x
        time_sur, h_plus_sur, h_cross_sur, amp_sur = EvaluateSurrogate(sur, surrogate_dict)
        timeH_sur, strainH_sur, timeL_sur, strainL_sur = ProjectGivenGeocenterTime(time_sur, h_plus_sur, h_cross_sur, surrogate_dict)
        #ov = ComputeMultiDetectorOverlap(timeH, strainH, timeL, strainL, \
        #                        timeH_sur, strainH_sur, timeL_sur, strainL_sur)
        ov = ComputeOverlap(timeH, strainH, timeH_sur, strainH_sur)
    
        ## 1 minus the overlap since we're doing minimization
        #print(1.0 - ov),
        return 1.0 - ov
    
    ## iterate from zero to 2 * pi
    res = minimize_scalar(f, bounds=(35.0, 40.0), method = 'bounded')
    return(res.x)

### Method for doing a full Bilby waveform reconstruction

In [60]:
def BilbyReconstruction(p):
    """ Reconstruct Bilby waveforms from bilby run located in directory p """

    ## Read in bilby parameters
    bilby_result = GetBilbyParameters(p)
    bilby_dict = GetBilbyDict(bilby_result)
    
    ## f_ref optimization
    f_ref_opt = OptimizeFref(p, bilby_dict)
    bilby_dict['f_ref'] = f_ref_opt
    print('Bilby result phi:', bilby_dict['phi'], 'Bilby result pol:', bilby_dict['pol'])
    
    time_bil, h_plus_bil, h_cross_bil, amp_bil = EvaluateSurrogate(sur, bilby_dict) 
    timeH_bil, strainH_bil, timeL_bil, strainL_bil = ProjectGivenGeocenterTime(time_bil, h_plus_bil, h_cross_bil, bilby_dict)

    return timeH_bil, strainH_bil, timeL_bil, strainL_bil, bilby_dict

### Perform and plot reconstructions

In [ ]:
def PlotReconstructions():
    """ Plot Bilby reconstructed waveforms for a variety of Bilby runs"""
    
    base_dir = '/home/maria.okounkova/BeyondGRAnalysis/BilbyPE/'
    SNR = '75'
    
    ## Grab the mass from the truth dictionary
    p_gr = base_dir + 'NoTilt_dCS_0p0_' + SNR + '/'
    TruthDict = GetInjectedParameters(p_gr)
    mass = TruthDict['mass']
    print("True polarization and phase: ", TruthDict['phi'], TruthDict['pol'])
    
    plt.figure(figsize=(10,5))
    
    #ells = ['0p0', '0p2', '0p25','0p3', '0p35', '0p37']
    ells = ['0p0', '0p2', '0p37']
    cs = ['black', 'orange'] + sns.color_palette('magma', n_colors=len(ells))
    
    for ell, color in zip(ells, cs):

        p = base_dir + 'NoTilt_dCS_' + ell + '_' + SNR + '/'
        ## Injected data
        timeH, strainH, timeL, strainL = GetFramesData(p)
        ## GR attempt at reconstructing data
        timeH_bil, strainH_bil, timeL_bil, strainL_bil, bilby_dict = BilbyReconstruction(p)

        plt.plot(timeH, strainH, lw = 0.5, color = color, ls = '-', label = '$\ell = %.1f$' % EllinKm(ell, mass))
        plt.plot(timeH_bil, strainH_bil, lw = 2.0, color = color, alpha = 0.5,\
                 label = r'$a_1 = %.2f, a_2 = %.2f$' % (bilby_dict['a_1'][2], bilby_dict['a_2'][2]))

    plt.xlim(-0.1 + TruthDict['t_gps'], 0.02 + TruthDict['t_gps'])
    plt.xlabel('Time [sec]')
    plt.ylabel(r'$h(t)$ in Hanford')
    legend = plt.legend(fontsize=18, loc = 'lower left', frameon=True)
    plt.grid()
    plt.tight_layout()
    plt.savefig('BilbyReconstructions.pdf')
    plt.show()

    
PlotReconstructions()

True polarization and phase:  0 0.824043851821
Surrogate phase evaluation:  -1.848701026621205 f_low: 0.0 f_ref: 36.90983005625053
Surrogate phase evaluation:  -1.848701026621205 f_low: 0.0 f_ref: 38.09016994374947
Surrogate phase evaluation:  -1.848701026621205 f_low: 0.0 f_ref: 36.18033988749895
Surrogate phase evaluation:  -1.848701026621205 f_low: 0.0 f_ref: 37.12209638315182


### Plot multiple waveform reconstructions

In [ ]:
def PlotMultipleReconstructions():

    plt.figure(figsize=(16,8))
    
    base_dir = '/home/maria.okounkova/BeyondGRAnalysis/BilbyPE/'
    
    ## Injected parameters
    p = base_dir + 'NoTilt_dCS_0p37_75'
    truth_dict = GetInjectedParameters(p)
    print(truth_dict)
    timeH, strainH, timeL, strainL = GetFramesData(p)
    
    ## Plot multiple reconstructed waveforms
    bilby_results = GetMultipleBilbyParameters(p, 50)

    for bilby_result in bilby_results:
        
        bilby_dict = GetBilbyDict(bilby_result)
        bilby_dict['phi'] = bilby_dict['phi']
        time_bil, h_plus_bil, h_cross_bil, amp_bil = EvaluateSurrogate(sur, bilby_dict) 
        timeH_bil, strainH_bil, timeL_bil, strainL_bil = ProjectGivenGeocenterTime(time_bil, h_plus_bil, h_cross_bil, bilby_dict)

        plt.plot(timeH_bil, strainH_bil, lw = 0.5, color = 'gray', alpha = 0.2)

    ## Plot GR waveform   
    plt.plot(timeH, strainH, lw = 2.0, color = '#1ae863', label = 'Injection', ls = '--')
    
    plt.xlim(-0.15 + truth_dict['t_gps'], 0.033 + truth_dict['t_gps'])
    plt.xlabel('Time [sec]')
    plt.ylabel(r'$h(t)$ in Hanford')
    legend = plt.legend(fontsize=18, loc = 'lower left', frameon=True)
    plt.grid()
    plt.tight_layout()
    plt.savefig('BilbyMultipleReconstructions.pdf')
    plt.show()

#PlotMultipleReconstructions()

### Evaluate GR Surrogate with various spins

In an effort to understand why the Bilby results prefer higher spins for the beyond-GR case, see what happens when spins are increased for the surrogate at fixed mass ratio, chirp mass, tilts, SNR, etc

In [ ]:
def SurrogateVariousSpins():
    """ Plot surrogate waveforms for different values of the spins a_1 and a_2 while keeping the mass ratio constant
        and ensuring that chi_eff ~ 0"""
    
    
    ## Plot for the actual bilby spin values as they change
    ## Plot the actual Bilby reconstructions
    
    desired_snr = 75
    
    plt.figure(figsize=(14, 8))
    
    a_1s = [0.0, 0.25, 0.5, 0.75]
    cs = ['black'] + sns.color_palette('magma', n_colors=len(a_1s))
    
    for a_1, color in zip(a_1s, cs): 
    
        a_2 = a_1 / 1.2212532137858916 ## mass ratio
        
        params_dict = {'dt' : 1/2048, 't_gps' : 1126259462.0, 'peak_time_in_segment' : 8.0, 'segment_length' : 16.0, \
               'dist_mpc' : 400.0, 'mass' : 68.0, 'q' : 1.2212532137858916, \
               'a_1' : [0, 0, a_1], 'a_2' : [0,0, -a_2], \
               'f_low' : 25, 'theta' : pi, 'phi' : 0, \
               'ra' : 1.952318922, 'dec' : -1.26967171703, 'pol' : 0.824043851821, 'geocent_time': 1126259461.9841547, \
                'm_1': 37.38665093292035, 'm_2': 30.61334906707965, 'mass_ratio': 0.8188310079446952, 'chirp_mass': 29.422167356249002}
        
        
        chi_eff = EffectiveSpin(params_dict)
        print("Effective spin: ", chi_eff)
    
        time, h_plus, h_cross, amp = EvaluateSurrogate(sur, params_dict)
        
        ## Now compute the SNR in the detectors and scale accordingly
        current_dist = params_dict['dist_mpc']
        timeH, strainH, timeL, strainL = ProjectGivenGeocenterTime(time, h_plus, h_cross, params_dict)
        strainH, strainL, params_dict = TargetSNR(timeH, strainH, timeL, strainL, params_dict, desired_snr)
        updated_dist = params_dict['dist_mpc']
        #h_plus = h_plus * current_dist / updated_dist
        #h_cross = h_cross * current_dist / updated_dist

        plt.plot(timeH, strainH, label = '$a_1 = %.3f, a_2 = %.3f$' % (a_1, a_2), lw = 1.0, color = color)
        

    plt.xlim(-0.15 +  params_dict['t_gps'], 0.02 +  params_dict['t_gps'])
    plt.xlabel('Time [sec]')
    plt.ylabel(r'$h(t)$ in Hanford')
    legend = plt.legend(fontsize=24, loc = 'lower left', ncol=2, frameon=False)
    plt.grid()
    plt.show()
    
#SurrogateVariousSpins()